# Markdown Cleaner

處理流程：
- 讀取來源 Markdown（保留原標題層級 `#`）
- 依設定移除粗體、HTML 註解、統一標題為 `##`
- 另存為附加 `_無粗體` 字尾之檔案

可在下方「設定區」控制各功能是否啟用，預設全部開啟。
完成設定後執行主程式格即可產生新檔。

In [ ]:
# ---- 設定區：可依需求調整 ----
source_relative = Path('mkdocs/My_Notes/113憲判字11號判決/法官學院書面稿.md')

output_relative = Path('mkdocs/My_Notes/113憲判字11號判決/給老師的/遺產稅案例研討_法官學院書面稿_20251011.md')
remove_bold = True
remove_comments = True
normalize_headings = False

# --------------------------------


In [ ]:
from pathlib import Path
import re

BASE_DIR = Path.cwd()
possible_roots = [BASE_DIR, BASE_DIR / 'work'] + list(BASE_DIR.parents) + list((BASE_DIR / 'work').parents)

def resolve_path(relative):
    p = Path(relative)
    if p.is_absolute():
        return p if p.exists() else None
    for base in possible_roots:
        candidate = (base / p).resolve()
        if candidate.exists():
            return candidate
    return None

source_path = resolve_path(source_relative)
if source_path is None:
    raise FileNotFoundError(f'找不到來源檔案：{source_relative}')

output_path = (BASE_DIR / output_relative).resolve()
output_path.parent.mkdir(parents=True, exist_ok=True)

text = source_path.read_text(encoding='utf-8')
print(f'來源檔案：{source_path}')
print(f'原始字元數：{len(text):,}')

processed = text

if remove_bold:
    bold_pattern = re.compile(r'\*\*(.+?)\*\*', flags=re.DOTALL)
    processed = bold_pattern.sub(lambda m: m.group(1), processed)

if remove_comments:
    comment_pattern = re.compile(r'<!--.*?-->', flags=re.DOTALL)
    processed = comment_pattern.sub('', processed)

if normalize_headings:
    heading_pattern = re.compile(r'^(#+)\s*(.*)$', flags=re.MULTILINE)
    def normalize(match: re.Match) -> str:
        hashes, title = match.group(1), match.group(2).strip()
        if len(hashes) <= 1:
            return '# ' + title
        return '## ' + title
    processed = heading_pattern.sub(normalize, processed)

output_path.write_text(processed, encoding='utf-8')
print(f'已輸出：{output_path}')
print(f'新檔字元數：{len(processed):,}')


來源檔案：/home/jovyan/work/mkdocs/My_Notes/113憲判字11號判決/法官學院書面稿.md
原始字元數：29,752
已輸出：/home/jovyan/mkdocs/My_Notes/113憲判字11號判決/給老師的/遺產稅案例研討_法官學院書面稿_20251011.md
新檔字元數：29,138
